In [2]:
import finlab
finlab.login(open("config.txt", "r").read())

from finlab import data
import pandas as pd
iStart = -250
opens    = data.get("price:開盤價").fillna(0).astype(float)[iStart: ]      #如果資料有空白自動填0
closes   = data.get("price:收盤價").fillna(0).astype(float)[iStart: ]
highs    = data.get("price:最高價").fillna(0).astype(float)[iStart: ]
lows     = data.get("price:最低價").fillna(0).astype(float)[iStart: ]
volumns  = data.get("price:成交股數").fillna(0).astype(float)[iStart: ]

ma5   = closes.ewm(span=5, adjust=False).mean().round(2)
ma7   = closes.ewm(span=7, adjust=False).mean().round(2)
ma10  = closes.ewm(span=10, adjust=False).mean().round(2)
ma20  = closes.ewm(span=20, adjust=False).mean().round(2)
ma60  = closes.ewm(span=60, adjust=False).mean().round(2)
ma240 = closes.ewm(span=240, adjust=False).mean().round(2)

kd_k, kd_s  = data.indicator('STOCH')[iStart: ] 
pd.set_option('display.expand_frame_repr', False)

輸入成功!


### 收盤後的跳空股票

In [3]:
import pandas as pd
import numpy as np
from finlab import data
from finlab import backtest
import _beowFmt as fm 

pd.set_option('display.expand_frame_repr', True)
ss = ''

yHigh = highs.shift(1)
yClose = closes.shift(1)
cond1 = opens - yHigh > 0
cond2 = closes >= opens
cond3 = (closes - opens)/opens > 0.02
cond4 = lows - yHigh > 0
cma2 = ma10 >= ma7
cma3 = ma20 >= ma10
cma4 = ma60 >= ma20
cma5 = ma240 >= ma60

position = cond1 & cond2 & cond3 & cond4 & cma2  & cma3  & cma4  & cma5 
dff = position.iloc[-1:].transpose()
dff["stock_ids"] = dff.index
dff.columns = ["val", "stock_ids"]
dff = dff[ (dff["stock_ids"].str.len() == 4) & (dff["val"] == True) ]
dff = dff.dropna(how='all')

today = position.iloc[-1:].index[0].strftime("%Y%m%d")

for c in dff["stock_ids"].tolist():
    ss += f"{c}.TW,"
fm.write_LogFile(f"xq-data/{today}_今天跳空開高.csv", ss) 


In [21]:
pd.set_option('display.expand_frame_repr', True)
ss = ''


cond1 = ((highs - closes) / (closes - opens)) > 2
cond2 = closes >= opens
cond3 = highs > closes
cond4 = closes > opens
cond5 = opens == lows
cond6 = opens > ma60

cma2 = ma10 >= ma7
cma3 = ma20 >= ma10
cma4 = ma60 >= ma20
cma5 = ma240 >= ma60


position = cond1 & cond2 & cond3 & cond4 & cond5 & cond6 #& cma5 
dff = position.iloc[-1:].transpose()
dff["stock_ids"] = dff.index
dff.columns = ["val", "stock_ids"]
dff = dff[ (dff["stock_ids"].str.len() == 4) & (dff["val"] == True) ]
dff = dff.dropna(how='all')

today = position.iloc[-1:].index[0].strftime("%Y%m%d")

for c in dff["stock_ids"].tolist():
    ss += f"{c}.TW,"
fm.write_LogFile(f"xq-data/{today}_紅K倒鎚子.csv", ss) 